In [1]:
from flask import Flask, request, render_template, jsonify
from flask_ngrok import run_with_ngrok
import cv2
from ultralytics import YOLO

In [2]:
app = Flask(__name__)
run_with_ngrok(app)
model = YOLO('yolov8n.pt')

In [3]:
COCO_CLASSES = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair",
    "couch", "potted plant", "bed", "dining table", "toilet", "TV", "laptop", "mouse",
    "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator",
    "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

In [4]:
def process_video(video_path, model):
    results = []
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    fps = cap.get(cv2.CAP_PROP_FPS) 

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        
        
        detections = model(frame)

        
        for det in detections[0].boxes:
            x_min, y_min, x_max, y_max = det.xyxy[0].tolist()  
            confidence = det.conf[0].item()  
            class_id = int(det.cls[0].item())  

            
            class_name = COCO_CLASSES[class_id]

           
            duration = frame_count / fps

            results.append({
                'class': class_name,
                'confidence': f"{confidence:.2f}",
                'x_min': f"{x_min:.1f}",
                'y_min': f"{y_min:.1f}",
                'x_max': f"{x_max:.1f}",
                'y_max': f"{y_max:.1f}",
                'duration': f"{duration:.2f}s"
            })
    
    cap.release()
    return results

In [5]:
def generate_copywriting(detected_objects):
    num_objects = len(detected_objects)
    copywriting = f"Captured a video with {num_objects} objects detected!\n"
    
    for obj in detected_objects:
        
        label = obj.get('class', '')
        confidence = obj.get('confidence', 0.0)
        location = (obj.get('x_min', 0), obj.get('y_min', 0), obj.get('x_max', 0), obj.get('y_max', 0))
        duration = obj.get('duration', 0)

        
        copywriting += (
            f"Detected object: {label}, Confidence: {confidence}, "
            f"Location: {location}, Duration: {duration}\n"
        )
    
    return copywriting

In [6]:

index_html_content = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Video Upload</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='styles.css') }}">
</head>
<body>
    <div class="container">
        <h1>Welcome! You are Using Automated Copywriting Services. Let's Get Started!</h1>
        <h2>Upload Video for Analysis</h2>
        <form action="/analyze_video" method="post" enctype="multipart/form-data">
            <input type="file" name="video" accept="video/*" required>
            <button type="submit">Upload and Analyze</button>
        </form>
    </div>
</body>
</html>
'''

with open('templates/index.html', 'w') as file:
    file.write(index_html_content)

print("index.html Created")

index.html Created


In [7]:
result_html_content = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Analysis Results</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='styles.css') }}">
</head>
<body>
    <div class="container">
        <h1>Analysis Results</h1>
        <h2>Detected Objects</h2>
        <ul>
            {% for detection in detections %}
                <li>
                    Object: {{ detection.class }}<br>
                    Confidence: {{ detection.confidence }}<br>
                    Location: ({{ detection.x_min }}, {{ detection.y_min }}, {{ detection.x_max }}, {{ detection.y_max }})<br>
                    Duration: {{ detection.duration }}
                </li>
            {% endfor %}
        </ul>
        <h2>Generated Copywriting</h2>
        <p>{{ copywriting }}</p>
        <a href="/" class="button">Upload Another Video</a>
    </div>
</body>
</html>
'''

with open('templates/result.html', 'w') as file:
    file.write(result_html_content)

print("result.html is Created.")

result.html is Created.


In [ ]:
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/analyze_video', methods=['POST'])
def analyze_video():
    if 'video' not in request.files:
        return jsonify({'error': 'No video file part'}), 400
    
    file = request.files['video']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400
    
    file_path = 'uploaded_video.mp4'
    file.save(file_path)

    
    detected_objects = process_video(file_path, model)
    copywriting = generate_copywriting(detected_objects)

    return render_template('result.html', detections=detected_objects, copywriting=copywriting)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\jayas\anaconda3\Lib\site-packages\urllib3\connection.py", line 203, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jayas\anaconda3\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "C:\Users\jayas\anaconda3\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\jayas\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 791, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jayas\anaconda3\Lib\site-packages\


0: 384x640 6 persons, 1 bicycle, 9 cars, 1 motorcycle, 3 buss, 624.6ms
Speed: 20.8ms preprocess, 624.6ms inference, 28.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 1 bicycle, 6 cars, 1 motorcycle, 3 buss, 323.9ms
Speed: 11.1ms preprocess, 323.9ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 1 bicycle, 7 cars, 1 motorcycle, 3 buss, 288.0ms
Speed: 8.5ms preprocess, 288.0ms inference, 6.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 1 bicycle, 9 cars, 2 motorcycles, 3 buss, 432.4ms
Speed: 9.0ms preprocess, 432.4ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 1 bicycle, 11 cars, 1 motorcycle, 3 buss, 283.8ms
Speed: 7.0ms preprocess, 283.8ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 1 bicycle, 11 cars, 1 motorcycle, 3 buss, 274.4ms
Speed: 6.1ms preprocess, 274.4ms inference, 5.0ms postprocess per image 

127.0.0.1 - - [25/Aug/2024 16:04:43] "POST /analyze_video HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2024 16:04:43] "GET /static/styles.css HTTP/1.1" 404 -
